In [2]:
import glob,os
from nltk.tokenize import word_tokenize
import operator

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
path = './data'
data = []
for infile in glob.glob( os.path.join(path, '*.*') ):
       with open(infile) as file:
            string = file.readlines()
            for each_line in string:
                if len(each_line.split())>5: # to remove time and name from the doc
                    data.extend(each_line.replace('\n','').split(":")[2:])

In [4]:
# Importing files containing Ques, Ans and Others data
ques_cluster = open('./Clusters/query.txt','r').readlines()
ans_cluster = open('./Clusters/response.txt','r').readlines()
others_cluster = open('./Clusters/other.txt','r').readlines()

In [5]:
# List of all Concatenated Questions, Concatenated Answers, Concatenated Others
Questions_String = [" ".join([i for i in ques_cluster])]
Answers_String = [" ".join([i for i in ans_cluster])]
Others_String = [" ".join([i for i in others_cluster])]

# List of ques, ans and others
ques_list = [i for i in ques_cluster]
ans_list = [i for i in ans_cluster]
others_list = [i for i in others_cluster]

# List containing All concatenated Questions, list of Answers and List of Others
AllQuestion_ans_others = Questions_String + ans_list + others_list #[Q,A,O,A,O,A,A,O]
AllAnswer_ques_others = Answers_String + ques_list + others_list
AllOthers_ans_ques = Others_String + ans_list + ques_list

## Questions Feature

In [6]:
stop_words = ['i','me', 'my', 'myself','we','our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their','theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being','have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing','a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for','with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'all','any', 'both', 'each', 'few','more', 'most', 'other','some','such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will','just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn','weren','won','wouldn']

In [7]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(AllQuestion_ans_others).todense() #fit the vectorizer to synopses

In [8]:
ques_features = list(zip(tfidf_vectorizer.get_feature_names(), tfidf_matrix.tolist()[0]))
ques_features = sorted(ques_features, key=lambda x: x[1], reverse= True)
ques_features = [i for i in ques_features if i[1]>0]

## Answers Features

In [9]:
#define vectorizer parameters
tfidf_vectorizer_ans = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_ans = tfidf_vectorizer_ans.fit_transform(AllAnswer_ques_others).todense() #fit the vectorizer to synopses

In [10]:
ans_features = list(zip(tfidf_vectorizer_ans.get_feature_names(), tfidf_matrix_ans.tolist()[0]))
ans_features = sorted(ans_features, key=lambda x: x[1], reverse= True)
ans_features = [i for i in ans_features if i[1]>0]

## Others Features

In [11]:
#define vectorizer parameters
tfidf_vectorizer_others = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_others = tfidf_vectorizer_others.fit_transform(AllOthers_ans_ques).todense() #fit the vectorizer to synopses

In [12]:
others_features = list(zip(tfidf_vectorizer_others.get_feature_names(), tfidf_matrix_others.tolist()[0]))
others_features = sorted(others_features, key=lambda x: x[1], reverse= True)
others_features = [i for i in others_features if i[1]>0]

## All Features in Corpus 

In [13]:
all_features = [i[0] for i in ques_features] + [i[0] for i in ans_features] + [i[0] for i in others_features]
all_features = list(set(all_features))
all_features = [x for x in all_features if not x.strip().isdigit() ]

## Frequency Matrix

In [14]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(all_features)
#print("Vocabulary:", count_vectorizer.vocabulary_)

<284x284 sparse matrix of type '<class 'numpy.int64'>'
	with 284 stored elements in Compressed Sparse Row format>

In [15]:
train_data = ques_list+ans_list+others_list

In [15]:
train_data, test_data, class_list_train, class_list_test = train_test(ques_list, ans_list, others_list, 0.3)

In [16]:
class_list_train = ['Ques']*len(ques_list)+['Ans']*len(ans_list)+['Others']*len(others_list)

## Grid Search-SKLEARN to select HyperParameter

In [17]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators' : [50,100,150] ,'max_features' : ['auto',50,75,100,150] ,'max_depth' : [10,20,30,50],'max_leaf_nodes' : [3,7,9,12]}]

In [18]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_features='auto',max_depth=10, max_leaf_nodes=3)

In [20]:
CV_rfc = GridSearchCV(estimator=rf_classifier, param_grid = tuned_parameters, cv= 5)
CV_rfc.fit(count_vectorizer.transform(train_data), class_list_train)
print(CV_rfc.best_params_)

{'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': 9, 'n_estimators': 50}


In [31]:
y_pred = CV_rfc.predict(count_vectorizer.transform(data))
for i,j in enumerate(data):
    if y_pred[i] == 'Ques':
        print(j+"::::::::"+y_pred[i])

 My computer wont boot up and it goes to the home page and lets me put in my password but then it goes black and i cant do anything ::::::::Ques
 Last week, on September 13th, I made a system update and I guess that something went wrong during the update, because after that, when I turn the device on, it just shows me a black screen and I am not able to access any of my files from desktop… ::::::::Ques
 I tried to restart the computer and scan the system to check  if something is wrong… and it gives the message that it found some corrupt files, but was not able to fix some of them::::::::Ques
 After doing this step, even though the corrupt files are not solve, the screen is back to normal, but when I turn the laptop off and will turn it on again, the same problem happens!::::::::Ques


In [ ]:
for i, sentence in enumerate(test_data):
    if(CV_rfc.predict(count_vectorizer.transform([sentence])) == class_list_test[i]):
        print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

In [ ]:
for i, sentence in enumerate(test_data):
    if(CV_rfc.predict(count_vectorizer.transform([sentence])) != class_list_test[i]):
        print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

In [ ]:
ordered_features = [key for key, value in sorted(count_vectorizer.vocabulary_.items(), key=operator.itemgetter(1))]

In [ ]:
def get_class_features(list_of_sentences, count_vectorizer = count_vectorizer, ordered_features = ordered_features, rf = CV_rfc ):
    
    class_of = count_vectorizer.transform(list_of_sentences)
    class_is = rf.predict(class_of)
    
    features = []
    for each_class in class_of.todense().tolist():
        test = list(zip(ordered_features, each_class))
        features.append([i for i in test if i[1]>0])
    
    class_feature = {}
    for index, each_sentence in enumerate(list_of_sentences):
        class_feature.update({each_sentence:{"class": class_is[index], "feature": features[index]}})
    return class_feature

In [ ]:
input_document = ["Do i ask check the create this task with administrative privileges?", "My laptop is not working.","How are you?","While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy."]

In [ ]:
get_class_features(input_document)

In [1]:
a= {'as':1,'aq':2,'az':3}

In [2]:
a

{'aq': 2, 'as': 1, 'az': 3}

In [22]:
if 'ay' not in a.keys() or 'at' not in a.keys():
    print("true")

true
